In [5]:
import pandas as pd

# read in the main csv file
df = pd.read_csv("../all_raw_csv/filtered_file.csv")

# select 100 random rows from the dataframe
test_df = df.sample(n=700)

# write the selected rows to a new csv file
test_df.to_csv('../all_raw_csv/test_file.csv', index=False)


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import time

from tqdm import tqdm

# start the timer
start_time = time.time()

# read in the csv file
df = pd.read_csv("../all_raw_csv/filtered_file_no_images_no_duplicates.csv")

# specify a directory to save the images
save_dir = '../images/'

# create a list of indices of rows to be deleted
to_delete = []

# add a new column to the dataframe for the file name
df['file_name'] = None

# create a log file
with open("log_download_images.txt", "w") as log:
    log.write("Log File\n\n")
    log.write("Start Time: " + str(start_time) + "\n\n")

    # iterate through the rows of the dataframe
    for index, row in tqdm(df.iterrows(), total=len(df), desc='Downloading Images'):
        try:
            # get the url of the image
            url = row['url']
            response = requests.get(url, timeout=5)
            if response.status_code == 404:
                to_delete.append(index)
                log.write(f'{url} not found\n')
            else:
                # get the file name from the url
                file_name = url.split("/")[-1]
                # write the image to the specified directory
                open(save_dir + file_name, "wb").write(response.content)
                # update the relative_path column
                df.loc[index, 'relative_path'] = save_dir + file_name
                df.loc[index, 'file_name'] = file_name
                log.write(f'{file_name} downloaded\n')
        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError) as e:
            to_delete.append(index)
            log.write(f'Error: {e} for url {url}\n')
    end_time = time.time()
    log.write("\nEnd Time: " + str(end_time) + "\n")
    log.write(f'Time taken: {end_time - start_time} seconds\n')
    log.write(f'Images downloaded: {len(df) - len(to_delete)} \n')
    log.write(f'Images not found: {len(to_delete)} \n')


df.drop(to_delete, inplace=True)
df.to_csv('../all_csv_images/all_data_all_images_no_duplicates.csv', index=False)

In [4]:
df = pd.read_csv("../all_csv_images/all_data_all_images_no_duplicates.csv")

In [5]:
df.shape

(28109, 16)